<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=957b45eb80407aebd87c272ed2521d2187aa8fb1a765cf6b1947840569566f0f
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-16 12:25:02
-------------------
qualified stocks: 88
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  2.18 L
Current:  1.42 C
-------------------
Today PnL: 53.80 K (0.38%)
Current PnL: -17.00 L (-11.44%)
CY Booked + Current PnL: -5.97 L (-4.02%)
-------------------
Total profit:  2.26 L
Total loss:  -19.26 L
-------------------
Total Booked + Current PnL: 21.01 L (17.26%)
Total Booked PnL: 38.01 L (31.23%)
Curr Year Booked PnL: 11.03 L (7.76%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.25 C
Est FTT PnL: 82.90 L (58.3%)
Deployed:  1.22 C
Current:  1.42 C
CAGR/XIRR %: 9.67%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
63,SBIN,863.00,-11.34,65.0,M-LC,4.24,219712.0,18232.0,8986.0,0.53,9.05,4.09,13.51,60.0,2.03,1.57,22.79,XY25,NTT,BANKS
5,APOLLOHOSP,8285.00,-13.86,56.0,H-LC,11.04,172172.0,15995.0,10106.0,0.12,10.24,5.87,16.71,22.0,1.58,1.23,29.47,X40N,BTT,HEALTHCARE
78,TTKPRESTIG,770.00,104.27,58.0,M-SC,4.43,87966.0,-12811.0,12905.0,-0.10,-12.71,14.67,0.09,245.0,-0.99,0.63,15.57,OX40N,NTT,DURABLES
50,NATIONALUM,247.44,-42.83,70.0,H-MC,5.69,114560.0,14056.0,16588.0,-0.29,13.99,14.48,30.49,79.0,0.85,0.82,50.92,MH,ATH,METALS
42,ITC,452.00,-37.50,54.0,H-LC,2.86,202079.0,1941.0,18955.0,0.15,0.97,9.38,10.44,4.0,0.10,1.44,6.59,X40,NTT,FMCG


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
36,IEX,219.00,-29.93,54.0,H-SC,17.33,200049.0,95.0,97144.0,0.14,0.05,48.56,48.63,115.0,0.00,1.43,11.61,XR,NTT,MISC
35,ICICIPRULI,790.00,-20.65,45.0,H-MC,2.22,136628.0,840.0,41917.0,-0.11,0.62,30.68,31.48,107.0,0.02,0.98,12.81,X40,ATH,INSURANCE
12,BANKINDIA,190.00,-30.43,60.0,H-MC,5.81,180561.0,753.0,111659.0,-0.50,0.42,61.84,62.52,88.0,0.01,1.29,34.65,XR,NTT,BANKS
52,PGHH,17973.08,-31.08,50.0,H-MC,4.39,199425.0,-1395.0,70178.0,0.35,-0.69,35.19,34.25,80.0,-0.02,1.42,4.47,X40,ATH,FMCG


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,VBL,671.64,-17.18,36.0,H-LC,5.61,298265.0,-17577.0,130252.0,-0.88,-5.57,43.67,35.67,5.0,-0.13,2.13,7.52,X40N,ATH,FMCG
6,ASIANPAINT,3465.66,-12.80,47.0,H-LC,8.19,219991.0,-31777.0,84983.0,-0.11,-12.62,38.63,21.13,27.0,-0.37,1.57,18.33,X40,ATH,PAINTS
31,HCLTECH,1943.91,1.08,49.0,H-LC,10.02,230868.0,-11056.0,74316.0,0.27,-4.57,32.19,26.15,8.0,-0.15,1.65,9.10,X40,ATH,IT
20,CIPLA,1795.00,-20.12,50.0,H-LC,5.81,213336.0,8836.0,32576.0,0.57,4.32,15.27,20.25,10.0,0.27,1.52,13.43,X40N,BTT,PHARMA
81,UNITDSPR,1644.00,-13.23,51.0,H-LC,7.89,227298.0,-6846.0,55461.0,0.47,-2.92,24.40,20.77,86.0,-0.12,1.62,3.79,X40N,NTT,BREWERIES


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
48,LTIM,7230.20,-2.60,63.0,H-LC,1.01,241762.0,-4389.0,83601.0,0.58,-1.78,34.58,32.18,16.0,-0.05,1.73,33.62,X200,ATH,IT
42,ITC,452.00,-37.50,54.0,H-LC,2.86,202079.0,1941.0,18955.0,0.15,0.97,9.38,10.44,4.0,0.10,1.44,6.59,X40,NTT,FMCG
51,NESTLEIND,1377.00,-7.97,59.0,H-LC,3.79,283093.0,17667.0,39123.0,-0.22,6.66,13.82,21.40,11.0,0.45,2.02,13.74,XY25,NTT,FMCG
57,RELIANCE,1533.00,-12.54,56.0,H-LC,5.58,218884.0,8518.0,20269.0,0.27,4.05,9.26,13.68,37.0,0.42,1.56,21.24,XY25,NTT,REFINERIES
84,VBL,671.64,-17.18,36.0,H-LC,5.61,298265.0,-17577.0,130252.0,-0.88,-5.57,43.67,35.67,5.0,-0.13,2.13,7.52,X40N,ATH,FMCG


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VAIBHAVGBL,521.00,63.64,70.0,H-SC,8.52,142038.0,-40737.0,151810.0,5.82,-22.29,106.88,60.77,125.0,-0.27,1.01,27.22,XR,NTT,JEWELLERY
22,COFFEEDAY,80.00,-41.65,55.0,L-SC,41.96,87514.0,-26035.0,66091.0,2.61,-22.93,75.52,35.27,268.0,-0.39,0.63,114.19,XR,NTT,HOTELS
10,BAJAJHFL,152.00,-9.61,63.0,H-MC,6.05,190683.0,-9823.0,58597.0,2.58,-4.90,30.73,24.33,90.0,-0.17,1.36,9.96,X40N,BTT,FINANCE
55,RAJOOENG,143.33,-34.33,44.0,H-SC,10.65,99580.0,-2920.0,43745.0,2.45,-2.85,43.93,39.83,114.0,-0.07,0.71,4.09,AR,ATH,MISC
77,TRIDENT,48.00,-5.84,63.0,M-SC,5.37,73359.0,-18896.0,45204.0,2.41,-20.48,61.62,28.51,224.0,-0.42,0.52,25.53,XR,NTT,TEXTILES


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TITAGARH,1548.00,-0.57,65.0,H-SC,4.17,163188.0,-28827.0,107720.0,-1.67,-15.01,66.01,41.08,158.0,-0.27,1.17,38.28,XY24,NTT,ENGINEERING
65,SHALBY,327.00,1067.46,65.0,M-SC,2.56,158787.0,-22356.0,67818.0,-1.55,-12.34,42.71,25.10,235.0,-0.33,1.13,27.25,XY24,NTT,HEALTHCARE
4,ANGELONE,3033.00,3.06,35.0,H-SC,9.27,182630.0,-8376.0,66076.0,-1.10,-4.39,36.18,30.21,157.0,-0.13,1.30,15.58,X40N,NTT,FINANCE
27,GREENPANEL,537.00,233.98,64.0,M-SC,4.16,149011.0,-34067.0,113040.0,-0.97,-18.61,75.86,43.14,230.0,-0.30,1.06,40.17,XY24,NTT,MISC
84,VBL,671.64,-17.18,36.0,H-LC,5.61,298265.0,-17577.0,130252.0,-0.88,-5.57,43.67,35.67,5.0,-0.13,2.13,7.52,X40N,ATH,FMCG


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,UJJIVANSFB,60.0,98.37,59.0,H-SC,12.45,125469.0,-17010.0,36537.0,-0.54,-11.94,29.12,13.70,163.0,-0.47,0.90,48.09,OX40N,NTT,BANKS
78,TTKPRESTIG,770.0,104.27,58.0,M-SC,4.43,87966.0,-12811.0,12905.0,-0.10,-12.71,14.67,0.09,245.0,-0.99,0.63,15.57,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-67.61,59.0,H-SC,1.74,223191.0,-46476.0,82804.0,0.75,-17.23,37.10,13.47,138.0,-0.56,1.59,12.89,XY24,NTT,PAINTS
19,CERA,9475.0,-20.05,45.0,H-SC,2.16,145556.0,-30347.0,72370.0,0.15,-17.25,49.72,23.89,149.0,-0.42,1.04,25.71,OX40N,NTT,CERAMICS
73,TATAELXSI,9161.0,-20.01,52.0,H-MC,7.81,108167.0,-23668.0,65895.0,0.14,-17.95,60.92,32.03,98.0,-0.36,0.77,21.16,OX40N,NTT,IT


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TCS,4476.75,-26.12,55.0,H-LC,12.82,294784.0,-51172.0,126020.0,0.77,-14.79,42.75,21.64,1.0,-0.41,2.11,4.43,X40,ATH,IT
40,INFY,2275.00,-17.80,52.0,H-LC,8.81,320906.0,8018.0,163662.0,-0.12,2.56,51.00,54.87,3.0,0.05,2.29,9.38,X40,BTT,IT
42,ITC,452.00,-37.50,54.0,H-LC,2.86,202079.0,1941.0,18955.0,0.15,0.97,9.38,10.44,4.0,0.10,1.44,6.59,X40,NTT,FMCG
84,VBL,671.64,-17.18,36.0,H-LC,5.61,298265.0,-17577.0,130252.0,-0.88,-5.57,43.67,35.67,5.0,-0.13,2.13,7.52,X40N,ATH,FMCG
1,ABB,7934.00,-38.50,63.0,H-LC,11.12,257064.0,-4555.0,123776.0,0.31,-1.74,48.15,45.57,7.0,-0.04,1.84,9.38,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,45.15,65.0,M-MC,6.97,228905.0,3943.0,163713.0,0.11,1.75,71.52,74.53,192.0,0.02,1.63,34.57,XY24,BTT,STEEL
15,BLUESTARCO,2326.38,12.08,61.0,H-MC,3.71,193350.0,28680.0,39289.0,2.35,17.42,20.32,41.28,89.0,0.73,1.38,26.79,X40N,ATH,AC
85,VOLTAS,1918.49,1.72,61.0,H-MC,2.67,213435.0,21693.0,74339.0,0.50,11.31,34.83,50.08,99.0,0.29,1.52,18.88,XY25,ATH,AC
30,HAVELLS,2069.16,1.33,65.0,H-MC,1.82,251082.0,3351.0,71709.0,0.90,1.35,28.56,30.30,92.0,0.05,1.79,14.82,X40,ATH,ELECTRICAL
24,DABUR,735.00,-3.30,57.0,H-MC,6.57,210581.0,13553.0,76062.0,-0.18,6.88,36.12,45.49,102.0,0.18,1.50,20.82,XY24,BTT,FMCG


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,247.44,-42.83,70.0,H-MC,5.69,114560.0,14056.0,16588.0,-0.29,13.99,14.48,30.49,79.0,0.85,0.82,50.92,MH,ATH,METALS
86,WHIRLPOOL,2270.00,-39.28,59.0,M-SC,0.62,102720.0,11222.0,67528.0,1.94,12.26,65.74,86.07,223.0,0.17,0.73,49.17,XR,NTT,DURABLES
37,INDIAMART,4850.92,-50.72,52.0,H-SC,12.20,137758.0,14422.0,119340.0,-0.38,11.69,86.63,108.45,119.0,0.12,0.98,36.66,AR,ATH,MISC
87,WIPRO,420.00,-11.07,54.0,M-LC,5.60,156271.0,5326.0,104123.0,0.31,3.53,66.63,72.51,53.0,0.05,1.12,9.76,XR,NTT,IT
12,BANKINDIA,190.00,-30.43,60.0,H-MC,5.81,180561.0,753.0,111659.0,-0.50,0.42,61.84,62.52,88.0,0.01,1.29,34.65,XR,NTT,BANKS


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,80.00,-41.65,55.0,L-SC,41.96,87514.0,-26035.0,66091.0,2.61,-22.93,75.52,35.27,268.0,-0.39,0.63,114.19,XR,NTT,HOTELS
3,ALKYLAMINE,4546.37,-4.34,48.0,H-SC,13.34,89053.0,-11910.0,106445.0,0.47,-11.80,119.53,93.63,148.0,-0.11,0.64,32.64,SR,ATH,CHEMICALS
49,MASFIN,398.61,-14.63,61.0,H-SC,12.95,96795.0,-1185.0,22786.0,0.86,-1.21,23.54,22.05,152.0,-0.05,0.69,40.70,XR,ATH,FINANCE
50,NATIONALUM,247.44,-42.83,70.0,H-MC,5.69,114560.0,14056.0,16588.0,-0.29,13.99,14.48,30.49,79.0,0.85,0.82,50.92,MH,ATH,METALS
8,ATULAUTO,844.00,3905.41,65.0,M-SC,6.45,124938.0,-22430.0,74251.0,2.23,-15.22,59.43,35.16,236.0,-0.30,0.89,27.61,XY24,NTT,AUTO


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,ATULAUTO,844.00,3905.41,65.0,M-SC,6.45,124938.0,-22430.0,74251.0,2.23,-15.22,59.43,35.16,236.0,-0.30,0.89,27.61,XY24,NTT,AUTO
22,COFFEEDAY,80.00,-41.65,55.0,L-SC,41.96,87514.0,-26035.0,66091.0,2.61,-22.93,75.52,35.27,268.0,-0.39,0.63,114.19,XR,NTT,HOTELS
61,SAMMAANCAP,326.00,-170.45,62.0,M-SC,2.63,81738.0,-20472.0,113861.0,0.31,-20.03,139.30,91.37,208.0,-0.18,0.58,32.69,XY25,NTT,FINANCE
50,NATIONALUM,247.44,-42.83,70.0,H-MC,5.69,114560.0,14056.0,16588.0,-0.29,13.99,14.48,30.49,79.0,0.85,0.82,50.92,MH,ATH,METALS
82,VAIBHAVGBL,521.00,63.64,70.0,H-SC,8.52,142038.0,-40737.0,151810.0,5.82,-22.29,106.88,60.77,125.0,-0.27,1.01,27.22,XR,NTT,JEWELLERY


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.49
1,25,42.74
2,50,73.11


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.19
LC    33.97
MC    22.77
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.46
X40      15.01
X40N     12.75
XR       11.39
XY25      9.62
AR        8.50
OX40N     8.06
X200      1.73
MH        1.70
X5K       1.44
SR        1.27
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    28.34
H-LC    25.84
H-MC    19.86
M-SC    13.32
M-LC     7.08
M-MC     2.61
L-SC     1.53
L-LC     1.05
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
IT,13.14,-14.27,76.58
FMCG,11.60,-4.12,40.56
FINANCE,9.49,-15.72,61.04
BANKS,7.63,-12.28,63.98
MISC,7.34,-11.52,73.54
PAINTS,5.76,-14.37,31.42
ELECTRICAL,5.60,-4.99,62.85
AC,3.60,4.01,28.49
INSURANCE,3.36,-7.11,43.85


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2856141.0,22
XR,1312948.0,14
AR,1303860.0,9
X40,746588.0,10
X40N,624084.0,10
OX40N,519743.0,10
XY25,404059.0,7
SR,244434.0,2
X5K,116160.0,1


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3441612.0,29
M-SC,1375570.0,17
H-LC,1256149.0,15
H-MC,1170678.0,15
M-LC,388602.0,5
M-MC,312291.0,2
L-SC,246914.0,3
L-MC,60125.0,1
L-LC,37847.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1042816.0      6
M-SC       XY24       796765.0      7
H-SC       AR         785080.0      5
           XR         761188.0      7
H-MC       XY24       484832.0      4
H-LC       X40        467936.0      5
           AR         356721.0      2
M-MC       XY24       312291.0      2
H-SC       X40N       297803.0      4
           OX40N      248709.0      4
           SR         244434.0      2
H-LC       X40N       228395.0      4
H-MC       X40        209845.0      4
           XY25       183973.0      2
L-SC       XR         163121.0      2
M-SC       AR         162059.0      2
M-LC       XY24       159333.0      2
M-SC       OX40N      121346.0      4
M-LC       X5K        116160.0      1
M-SC       XY25       113861.0      1
           XR         112732.0      2
H-MC       XR         111659.0      1
M-LC       XR         104123.0      1
H-MC       X40N        97886.0      2
L-SC       OX40N       83793.0      1
H-LC       X200        83601.0      1
M-SC       X40         68807.0      1
H-MC       OX40N       65895.0      1
H-SC       MH          61582.0      1
L-MC       XR          60125.0      1
H-LC       XY24        60104.0      1
           XY25        59392.0      2
L-LC       XY25        37847.0      1
H-MC       MH          16588.0      1
M-LC       XY25         8986.0      1

In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 69.0 seconds
